In [ ]:
from pyteomics import mgf
import pandas as pd
import numpy as np
import glob as glob
import os
import matplotlib.pyplot as plt
import networkx as nx
import pingouin as pg
import seaborn as sns
sys.path.insert(0,'/global/homes/b/bpb/repos/metatlas')
from metatlas.io import feature_tools as ft

In [ ]:
df = pd.read_csv('../data/public_and_internal_files_with_massive_and_redu.tsv', sep='\t')

df = df[~df['buddy'].str.contains('qc',case=False)]
df = df[~df['buddy'].str.contains('blank',case=False)]
# df['keywords'] = df['keywords'].apply(lambda x: x.split('###') if type(x)==str else [])
# df['keyword_DOM'] = df['keywords'].apply(lambda x: True if (('dom' in x) | ('organic matter' in x) | ('soil' in x)) else False)
idx1 = (df['in_massive_dom_list'])# | df['keyword_DOM']
# idx2 = (df['SampleType']=='plant')

# idx2 = df['data_dir']=='/global/cfs/cdirs/metatlas/projects/massive_data_for_scn'
idx3 = df['data_dir']=='/global/cfs/cdirs/metatlas/projects/rawdata_for_scn'
# df = df[(idx3)]# | (idx1)] #  | (idx2)
df = df[(idx3) | (idx1)] #  | (idx2)
# df = df[df['SampleType']=='plant']
file_metadata = df.copy()


out_dir = '/global/cfs/cdirs/metatlas/projects/carbon_network/raw_data'
temp_files = df['h5'].tolist()
files = []
for f in temp_files:
    base_dir = os.path.dirname(f)
    base_name = os.path.basename(f)
    new_dir = os.path.join(out_dir,base_dir)
    new_name = os.path.join(new_dir,base_name)
    files.append(new_name)
    if not os.path.isfile(new_name):
        print('File Not Found!')
        print(new_name)
        print(f)
files = pd.DataFrame(files,columns=['filename'])
print(files.shape)


In [ ]:
# G = nx.read_graphml('../data/structural_clusters_network.graphml')
# G = nx.read_graphml('../data/network.graphml')
G = nx.read_graphml('/global/homes/b/bpb/repos/scndb/build/CarbonNetwork.graphml')
df = dict(G.nodes(data=True))
df = pd.DataFrame(df).T
df.index.name = 'node_id'
df.reset_index(inplace=True,drop=False)
df.head()
df.columns

In [ ]:
idx = pd.notna(df['pathway_results_propagated'])
sum(idx) - temp.shape[0]

In [ ]:
temp.shape[0]

In [ ]:
temp = df[['h_to_c','o_to_c','pathway_results','formula_identity','predicted_formula']]
temp = temp[pd.notna(temp['predicted_formula'])]
temp = temp[pd.notna(temp['h_to_c'])]
temp = temp[pd.notna(temp['o_to_c'])]
temp.fillna('Unknown',inplace=True)
temp.shape[0] / df.shape[0]

In [ ]:
idx = temp['predicted_formula']!=temp['formula_identity']
temp[idx].shape[0]/temp.shape[0]

In [ ]:


# Plot the dendrogram
plt.figure(figsize=(10, 10))
import matplotlib.pyplot as plt

# Define the colors for each category
# color_map = {'environment': 'red', 'plant': 'blue'}

# Map the colors to the 'origin' column
u = temp['pathway_results'].unique()

fig,ax = plt.subplots()
# Create a scatter plot of the PCA results
for uu in u:
    idx = temp['pathway_results']==uu
    ax.plot(temp.loc[idx,'o_to_c'], temp.loc[idx,'h_to_c'],'.',markersize=10,label=uu,alpha=0.7)
    
ax.set_xlim(0,1.2)
ax.set_ylim(0,2.5)
ax.legend(bbox_to_anchor=(1.01, 1.01),loc='upper left')
ax.set_xlabel('O/C ratio',fontsize=18)
ax.set_ylabel('H/C ratio',fontsize=18)


In [ ]:
429/(12*150)*1000

In [ ]:
sum(pd.notna(df['inchi_key_identity'])),sum(pd.notna(df['pathway_results'])),df.shape[0]

In [ ]:

ppm_tolerance = 5
mz_tol = 0.002
rt_min = 1
rt_max = 100

atlas = df[['node_id','precursor_mz']].copy()
atlas.rename(columns={'precursor_mz':'mz','node_id':'label'},inplace=True)
atlas['rt_min'] = rt_min
atlas['rt_max'] = rt_max
atlas['mz_tolerance'] = mz_tol
atlas['rt_tolerance'] = 100
atlas['ppm_tolerance'] = ppm_tolerance
atlas['extra_time'] = 0
atlas['rt_peak'] = (rt_min+rt_max)/2  
atlas['group_index'] = ft.group_consecutive(atlas['mz'].values[:],
                                    stepsize=ppm_tolerance,
                                    do_ppm=True)



In [ ]:

out = []
counter = 0
# files = files.sample(150)
for f in files['filename']:
    try:
        d = ft.get_atlas_data_from_file(f,atlas,desired_key='ms1_neg')
    except:
        print('Can not read',f)
        continue
    d = d[d['in_feature']==True].groupby('label').apply(ft.calculate_ms1_summary).reset_index()
    d['filename'] = f
    out.append(d)
    counter += 1
    print(counter)
out = pd.concat(out)
out.rename(columns={'label':'node_id'},inplace=True)



In [ ]:
# out.to_pickle('/global/cfs/cdirs/metatlas/projects/carbon_network/temp_data/all_berkeley_lab_data.pkl')
# out.to_pickle('/global/cfs/cdirs/metatlas/projects/carbon_network/temp_data/century_experiment_data.pkl')
# out.to_pickle('/global/cfs/cdirs/metatlas/projects/carbon_network/temp_data/wavestab3_data.pkl')
# out.to_pickle('/global/cfs/cdirs/metatlas/projects/carbon_network/temp_data/plantmasst_data.pkl')
out.to_pickle('/global/cfs/cdirs/metatlas/projects/carbon_network/temp_data/allCarbonData.pkl')

In [ ]:
# out = pd.read_pickle('/global/cfs/cdirs/metatlas/projects/carbon_network/temp_data/all_berkeley_lab_data.pkl')
# out = pd.read_pickle('/global/cfs/cdirs/metatlas/projects/carbon_network/temp_data/wavestab3_data.pkl')
# out = pd.read_pickle('/global/cfs/cdirs/metatlas/projects/carbon_network/temp_data/century_experiment_data.pkl')
out = pd.read_pickle('/global/cfs/cdirs/metatlas/projects/carbon_network/temp_data/allCarbonData.pkl')
out_plant = pd.read_pickle('/global/cfs/cdirs/metatlas/projects/carbon_network/temp_data/plantmasst_data.pkl')


In [ ]:
def make_spectrum(out):
    # y_col = 'peak_height'
    y_col = 'peak_area'
    cols = ['mz_centroid',y_col]
    # & (out['num_datapoints']>1) & (out['peak_height']<1e80) & (out['peak_height']>1)
    y = out[(out['mz_centroid']>150)].groupby('node_id')[cols].median()
    # y = y[y['peak_area']<1e9]
    y.reset_index(inplace=True,drop=True)
    fig, ax = plt.subplots(figsize=(15,4))
    ax.vlines(y['mz_centroid'],0*y[y_col],y[y_col])
    # ax.set_xlim(300.0,320)
    # ax.set_yscale('log')

make_spectrum(out)
make_spectrum(out_plant)

In [ ]:
out['origin'] = 'environment'
out_plant['origin'] = 'plant'
out = pd.concat([out,out_plant])

In [ ]:
origin = out[['filename','origin']].drop_duplicates()
origin

In [ ]:
import pandas as pd
import numpy as np

def make_dataframe(out,fill_value = 0):
    min_value = 0
    # out['sampletype'] = out['filename'].apply(lambda x: os.path.basename(x))
    node_data = pd.pivot_table(out,index=['node_id'],columns=['filename'],values='peak_area',aggfunc=np.mean,fill_value=fill_value)
    # cols = node_data.columns
    # node_data['lbnl-carbon'] = node_data.mean(axis=1)
    # Select the columns containing the intensity values

    # node_data['average_intensity'] = node_data.apply(lambda row: np.mean(row.nlargest(10)), axis=1)
    # node_data.drop(columns=cols,inplace=True)
    # node_data = node_data.apply(lambda x: np.log10(x))
    # Calculate the average of the 10 most intense columns for each row
    node_data = node_data.div(node_data.max())

    return node_data

carbon_df = make_dataframe(out,fill_value=0)
# plant_df = make_dataframe(out_plant,fill_value=0)



In [ ]:
plt.figure(ncols=2,nrows=1,figsize=(10, 10))
idx = carbon_df[carbon_df['origin']=='environment']

y = carbon_df[cols]


In [ ]:
temp = carbon_df.T
temp = pd.merge(temp,origin,left_index=True,right_on='filename')
temp.reset_index(inplace=True,drop=True)
temp.drop(columns=['filename'],inplace=True)
cols = temp.columns
cols = list(set(cols) - set(['origin']))
v = temp[cols].values
v[v<0.0001] = 0
v = v + 0.0001
v = np.log10(v)
Z = sch.linkage(v, method='ward')

# Plot the dendrogram
plt.figure(figsize=(10, 10))
import matplotlib.pyplot as plt

# Define the colors for each category
color_map = {'environment': 'red', 'plant': 'blue'}

# Map the colors to the 'origin' column
colors = temp['origin'].map(color_map)


from sklearn.decomposition import PCA
# Perform PCA on v dataframe


pca_result = pca.fit_transform(v)


fig,ax = plt.subplots()
# Create a scatter plot of the PCA results
s = ax.scatter(pca_result[:, 0], pca_result[:, 1],c=colors,alpha=0.7)
ax.legend()

In [ ]:
file_metadata.columns

In [ ]:
file_metadata = pd.read_csv('../data/public_and_internal_files_with_massive_and_redu.tsv', sep='\t')
file_metadata = file_metadata[['title','massive_id','h5','description','keywords']]
file_metadata.drop_duplicates(inplace=True)
file_metadata.reset_index(inplace=True,drop=True)
file_metadata

In [ ]:
cols = carbon_df.index.tolist()
g = pd.merge(carbon_df.T,file_metadata,left_index=True,right_on='h5',how='inner')[cols + ['massive_id']].groupby('massive_id').apply(lambda x: x.mean())
g = g > 0.01
g = g.sum(axis=1)
g = g.to_frame()
g.columns = ['num_nodes']
cols = ['title','massive_id','description','keywords']
g = pd.merge(g,file_metadata[cols],left_index=True,right_on='massive_id',how='inner').drop_duplicates().sort_values('num_nodes',ascending=False)
g.reset_index(inplace=True,drop=True)
# g.to_csv('node_count_per_experiment.csv',index=False)
g


In [ ]:
cols = carbon_df.index.tolist()
file_metadata.fillna('',inplace=True)
g = pd.merge(carbon_df.T,file_metadata,left_index=True,right_on='h5',how='inner')[cols + ['massive_id','title']].groupby(['massive_id','title']).apply(lambda x: x.mean())
g.shape

In [ ]:
g

In [ ]:
cols = carbon_df.index.tolist()
file_metadata.fillna('',inplace=True)
g = pd.merge(carbon_df.T,file_metadata,left_index=True,right_on='h5',how='inner')[cols + ['massive_id','title']].groupby(['massive_id','title']).apply(lambda x: x.mean())
g[g<0.0001] = 0
g = g + 0.001
g = np.log10(g)
print(g.shape)
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt

# Compute the linkage matrix
Z = sch.linkage(g.values, method='ward')

# Plot the dendrogram
plt.figure(figsize=(10, 14))
sch.dendrogram(Z, orientation='left', labels=g.index)

# Set plot title and labels
# plt.title('Vertical Dendrogram')
plt.xlabel('Distance')
plt.ylabel('Index')

# Show the plot
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.decomposition import PCA
from adjustText import adjust_text
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
# Perform PCA on the carbon_df dataframe


# pca = PCA(n_components=2)
cols = carbon_df.index.tolist()
file_metadata.fillna('',inplace=True)
g = pd.merge(carbon_df.T,file_metadata,left_index=True,right_on='h5',how='inner')[cols + ['massive_id','title']].groupby(['massive_id','title']).apply(lambda x: x.mean())


# pca_result = pca.fit_transform(g)

tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
pca_result = tsne.fit_transform(g)

fig,ax = plt.subplots()
# Create a scatter plot of the PCA results
ax.scatter(pca_result[:, 0], pca_result[:, 1])
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

# Create a list to hold the text objects
texts = []
for i, txt in enumerate(g.index.get_level_values(0)):
    texts.append(ax.text(pca_result[i, 0], pca_result[i, 1], txt, fontsize=9))
plt.show()


In [ ]:

fig,ax = plt.subplots(figsize=(20,20))
# Create a scatter plot of the PCA results
ax.scatter(pca_result[:, 0], pca_result[:, 1])
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

# Create a list to hold the text objects
texts = []
a1 = g.index.get_level_values(0)
a2 = g.index.get_level_values(1)
for i, txt in enumerate(zip(a1,a2)):
    texts.append(ax.text(pca_result[i, 0], pca_result[i, 1], txt, fontsize=9))
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create the clustermap
carbon_df = carbon_df + 0.0001
clustermap = sns.clustermap(carbon_df.apply(np.log10))

# Display the clustermap
plt.show()


In [ ]:
carbon_df.max(axis=1)

In [ ]:
nodes = [3061,2093,7579]
nodes = ['%.1f'%n for n in nodes]
carbon_df['sum'] = carbon_df.max(axis=1)
carbon_df.loc[carbon_df.index.isin(nodes),'sum']

In [ ]:
node_count = (carbon_df>0.01).sum(axis=1)
node_count = node_count.to_frame()
node_count.columns = ['node_count']

backup_G = G.copy()
nx.set_node_attributes(backup_G, node_count.to_dict('index'))
nx.write_graphml(backup_G,'../data/carbon_networkwith-CarbonSample-node-count.graphml')

In [ ]:
# min_value = 3000
# out['sampletype'] = 'plantmasst'
# node_data = pd.pivot_table(out,index=['node_id'],columns=['sampletype'],values='peak_area',aggfunc=np.mean,fill_value=min_value)
# node_data = node_data.apply(lambda x: np.log10(x))

# # node_data.columns = ['Quant: %s'%c for c in node_data.columns]
# # node_data = node_data[cols]
# # # Select the columns to normalize
# # columns_to_normalize = node_data.columns

# # # Normalize the selected columns
# # node_data[columns_to_normalize] = normalize(node_data[columns_to_normalize], norm='l2', axis=1)

backup_G = G.copy()
nx.set_node_attributes(backup_G, node_data.to_dict('index'))
nx.write_graphml(backup_G,'../data/carbon_networkwith-plantmasst.graphml')
backup_G_mst = nx.maximum_spanning_tree(backup_G)
nx.write_graphml(backup_G_mst,'../data/carbon_networkwith-plantmasst-mst.graphml')

In [ ]:
original_index = nx.get_node_attributes(G, 'original_index')
np_classifier_pathway = nx.get_node_attributes(G, 'pathway_results')
np_classifier_class = nx.get_node_attributes(G, 'class_results')
smiles_identity = nx.get_node_attributes(G, 'smiles_identity')
# structural_cluster_subclassname  = nx.get_node_attributes(G, 'structural_cluster_subclassname')
# structural_cluster_subclassname

In [ ]:
# node_data = pd.pivot_table(out,index=['node_id'],columns=['experiment'],values='peak_area',aggfunc=np.mean,fill_value=300)
# node_data = pd.pivot_table(out,index=['node_id'],columns=['sampletype'],values='peak_area',aggfunc=np.mean,fill_value=300)
# node_data = pd.pivot_table(out[out['sampletype'].str.contains('Hi')],index=['node_id'],columns=['experiment','sampletype'],values='peak_area',aggfunc=np.mean,fill_value=300)
# node_data.columns = ['%s-%s'%(c[1],c[0]) for c in node_data.columns]

# node_data = node_data.apply(lambda  x: np.log10(x+1),axis=1)
# s = node_data.sum(axis=0).values
# m = s.mean()
# node_data.values[:,:] = m * (node_data.values[:,:]/s)
# node_data.columns = [c.split('/')[-1].split('_')[12] for c in node_data.columns]

# node_data.to_csv('../data/log10_averages_treatments.csv')
node_data.columns = ['Quant: %s'%c for c in node_data.columns]

n = node_data.copy()
m = n.min(axis=1)
m = n.values - m.values[:,None]
n = pd.DataFrame(m,index=n.index,columns=n.columns)

m = n.max(axis=1)
m = n.values / m.values[:,None]
n = pd.DataFrame(m,index=n.index,columns=n.columns)


original_index = nx.get_node_attributes(G, 'original_index')
np_classifier_pathway = nx.get_node_attributes(G, 'pathway_results')
np_classifier_class = nx.get_node_attributes(G, 'class_results')
smiles_identity = nx.get_node_attributes(G, 'smiles_identity')
# temp = {}
# for c in n.columns:
    # temp[c] = nx.get_node_attributes(backup_G, c)

temp = pd.merge(n,pd.DataFrame({'np_classifier_pathway':np_classifier_pathway}),left_index=True,right_index=True)
temp = pd.merge(temp,pd.DataFrame({'np_classifier_class':np_classifier_class}),left_index=True,right_index=True)
temp = pd.merge(temp,pd.DataFrame({'smiles_identity':smiles_identity}),left_index=True,right_index=True)
cols = [c for c in temp.columns if 'Quant' in c]

# cluster_df = temp.groupby(['np_classifier_pathway'])[cols].mean()
cluster_df = temp.groupby(['np_classifier_class','np_classifier_pathway'])[cols].mean()
identity_df = temp.groupby(['np_classifier_pathway','np_classifier_class','smiles_identity'])[cols].mean()
cluster_df

In [ ]:
temp.sort_values('np_classifier_pathway').head(40)

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from rdkit import Chem
from rdkit.Chem import Draw
from scipy.cluster.hierarchy import linkage, dendrogram
# Create the polar plot

def order_similar_rows(cluster_df):
    # Calculate the pairwise distances between rows
    # cols = [c for c in cluster_df.columns]# if ('deciduousforests' in c) | ('coniferousforests' in c)]
    distances = cluster_df.values

    # Perform hierarchical clustering
    Z = linkage(distances, method='average', metric='euclidean')

    # Get the order of the rows based on the clustering
    order = dendrogram(Z, no_plot=True)['leaves']

    # Reorder the rows of cluster_df
    cluster_df = cluster_df.iloc[order]

    # Show the reordered cluster_df
    return cluster_df#[cols]

def make_polar_structural_cluster_plot(cluster_df):
    cluster_df = order_similar_rows(cluster_df)
    s = cluster_df.shape[0]/20*7 + 5
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(s, s), subplot_kw={'projection': 'polar'})

    # Define the angles for each side of the polygons
    angles = np.linspace(0, 2 * np.pi, cluster_df.shape[0] + 1)[:-1]
    shift_amount = np.diff(angles)[0] / cluster_df.shape[1]
    for iii in range(cluster_df.shape[1]):
        # Define the lengths of the bars
        bar_lengths = cluster_df.values[:, iii]
        # Plot the bars
        ax.bar(angles + iii * shift_amount, bar_lengths*1, width=shift_amount, align='edge', alpha=0.74, label=cluster_df.columns[iii].replace('Quant: ',''))

    # Set the labels for each side of the polygons
    ax.set_xticks(angles)
    ax.set_xticklabels(['' for i in range(cluster_df.shape[0])])

    # Set the title of the plot
    tick_labels = ax.xaxis.get_ticklabels()

    m = ax.get_ylim()[1]*1.1
    ax.set_ylim(0, m)



    counter = 0
    for i, row in cluster_df.iterrows():
        my_angle = angles[counter] + shift_amount* cluster_df.shape[1]/2
        # my_angle = angles[counter] + shift_amount*2
        # my_angle = my_angle * 180/np.pi - 90
        # my_angle = my_angle * 180/np.pi
        my_angle = 0
        # ax.text(angles[counter] + shift_amount*cluster_df.shape[1]/2, m * 1.11, '%s'%i, ha='center', va='center', fontsize=12, rotation=my_angle)#angles[counter]*180/4/np.pi)
        my_text = '%s'%i
        my_text = my_text.replace(' ','\n')
        ax.text(angles[counter] + shift_amount*cluster_df.shape[1]/2, m * 0.95, my_text, ha='center', va='center', fontsize=12, rotation=my_angle)#angles[counter]*180/4/np.pi)


        counter += 1

    ax.spines['polar'].set_visible(False)
    ax.grid(color='black')
    # Hide the y tick labels
    ax.set_yticklabels([])

    # Position the legend outside of the plot area
    ax.legend(bbox_to_anchor=(0.85, 1.1), loc='upper left',fontsize=16,frameon=False)

    plt.tight_layout()


unique_values = cluster_df.index.get_level_values('np_classifier_pathway').unique()
# unique_values = [u for u in unique_values is u!=None]
for uv in unique_values:
    selected_rows = cluster_df.loc[cluster_df.index.get_level_values('np_classifier_pathway') == uv]
    selected_rows.reset_index(drop=True, level="np_classifier_pathway", inplace=True)
    make_polar_structural_cluster_plot(selected_rows)
    # plt.savefig('../data/structural_clusters_%s.png'%uv)
    # plt.close()


In [ ]:
unique_values

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the number of sides of the polygons
num_sides = bar_df.shape[0]

# Define the angles for each side of the polygons
angles = np.linspace(0, 2 * np.pi, num_sides + 1)[:-1]



# Create the polar plot
fig, ax = plt.subplots(nrows=2,ncols=2,figsize=(12,12),subplot_kw={'projection': 'polar'})
ax = ax.flatten()

for i in range(len(ax)):
    # Define the lengths of the bars
    bar_lengths = bar_df.values[:,i]
    # Plot the bars
    ax[i].bar(angles, bar_lengths, width=0.1, align='edge', color='blue', alpha=0.5)


    # Set the labels for each side of the polygons
    ax[i].set_xticks(angles)
    ax[i].set_xticklabels(['Side {}'.format(i+1) for i in range(num_sides)])

    # Set the title of the plot
    ax[i].set_title(bar_df.columns[i])

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data
categories = ['Category 1', 'Category 2', 'Category 3', 'Category 4', 'Category 5']
values = [10, 15, 7, 12, 9]

# Convert values to radians
theta = np.linspace(0.0, 2 * np.pi, len(categories), endpoint=False)

# Create polar plot
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
bars = ax.bar(theta, values)

# Set the color of each bar
for bar in bars:
    bar.set_facecolor(np.random.rand(3))

# Set the labels for each category
ax.set_xticks(theta)
ax.set_xticklabels(categories)

# Set the title of the plot
ax.set_title('Circular Barchart')

# Show the plot
plt.show()
